In [1]:
import pandas as pd
import numpy as np
import os

In [39]:
#eval_df = pd.read_csv('/ceph/dpandya/notsofar/nsfd_adap_segments/adapter_base/large/eval_seqBN_r4full_ds_.csv')
eval_df = pd.read_csv('/ceph/dpandya/notsofar/nsfd_adap_segments/adapter_base/large/eval_parBN_r4full_ds_5e_.csv')
eval_df = eval_df.sort_values(by='wer', ascending=False)

In [40]:
eval_df['wer'].mean()

0.5522355068363344

In [41]:
preds = eval_df[eval_df.columns[-2]]
truth = eval_df[eval_df.columns[-3]]
wer = eval_df[eval_df.columns[-1]]

In [42]:
preds.iloc[0]

'And theyre unhappy about it And uh no So come uh you come with solutions you know what we come with a problem and you can come maybe come back and then well FILLlaugh consider your solutions What do you expect Mmhmm I mean you have a adjacents I mean weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since

In [43]:
eval_df['audio'] = eval_df['segmented_audio_file']

In [45]:
eval_df['preds'] = preds

In [46]:
eval_df['preds'].iloc[0]

'And theyre unhappy about it And uh no So come uh you come with solutions you know what we come with a problem and you can come maybe come back and then well FILLlaugh consider your solutions What do you expect Mmhmm I mean you have a adjacents I mean weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since the day of the elections weve had uh since

In [47]:
from transformers import WhisperForConditionalGeneration
import adapters

whisper_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")

whisper_adapter = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3")
adapters.init(whisper_adapter)
adapter_name = whisper_adapter.load_adapter('/ceph/dpandya/notsofar/nsfd_adap_segments/adapter_base/large/full_ds/parBN_r4full_ds_5e')
whisper_adapter.set_active_adapters(adapter_name)

There are adapters available but none are activated for the forward pass.


In [48]:
audio_files = eval_df['segmented_audio_file']

In [49]:
from transformers import AutomaticSpeechRecognitionPipeline
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained('openai/whisper-large-v3', language="english", task="transcribe")
eval_pipeline = AutomaticSpeechRecognitionPipeline(
            model=whisper_model,
            feature_extractor=processor.feature_extractor,
            tokenizer=processor.tokenizer,
            device="cuda:0", # Use GPU 0, or -1 for CPU
            chunk_length_s=30
        )

eval_adap_pipeline = AutomaticSpeechRecognitionPipeline(
            model=whisper_adapter,
            feature_extractor=processor.feature_extractor,
            tokenizer=processor.tokenizer,
            device="cuda:0", # Use GPU 0, or -1 for CPU
            chunk_length_s=30
        )

forced_decoder_ids = processor.get_decoder_prompt_ids(language="english", task="transcribe")

Device set to use cuda:0
Device set to use cuda:0


In [76]:
i = 3
pred = eval_pipeline(eval_df['audio'].iloc[i], 
              generate_kwargs = {'forced_decoder_ids': forced_decoder_ids,
                                 "temperature": 0.0})

/work/dpandya/miniconda3/envs/onlyWhisper/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [77]:
pred_adapter = eval_adap_pipeline(eval_df['audio'].iloc[i], 
              generate_kwargs = {'forced_decoder_ids': forced_decoder_ids,
                                 "temperature": 0.0,
                                 "no_repeat_ngram_size": 4})

/work/dpandya/miniconda3/envs/onlyWhisper/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [78]:
import jiwer
import eval_utils as eu

print(jiwer.wer(hypothesis=eu.clean_text(pred['text']), truth=eu.clean_text(eval_df["transcription"].iloc[i])), 
      jiwer.wer(hypothesis=eu.clean_text(pred_adapter['text']), truth=eu.clean_text(eval_df["transcription"].iloc[i])))

0.22058823529411764 0.4117647058823529


In [75]:
eu.clean_text(pred['text']), eu.clean_text(pred_adapter['text']), eu.clean_text(eval_df["transcription"].iloc[i])

('are they fine are they okay theyre great no no we didnt get much further ahead nothing substantial just discussing cats cats and dogs yeah okay so what i want to know is okay the company were renting the van from why are we renting well',
 'are they fine are they ok they can be graved ok thats it no no no we didnt get much further ahead no no we didnt give them much further ahead we were just discussing cats yeah um cats and dogs yeah ok so what iststst ststlaughs what i want to ststst what i want to know is um ok the stst laughs the company the filllaughs the company we were renting the van from why are we renting um well',
 'everything fine everything ok great no no uh no no we didnt get much further ahead nothing substantial yeah were just discussing cats um cats and dogs yeah ok so what i what i want to know is um ok the the the company uh the company we were renting the van from why are we renting uh well')

In [125]:
eval_df['preds_text_seqBN_r4full_ds_5e'].iloc[i], eval_df['wer'].iloc[i]

('Well I love what that sounds like but Im not sure that it would be practical for us to go that far So Ive put it into a different frame And Ive got the graphs for that Ill share that with you next week So that everyone can see how its impacting the overall business and the projected budgets for next year and for the year after',
 0.0149253731343283)

In [132]:
pd.read_csv('/ceph/dpandya/notsofar/nsfd_adap_segments/adapter_base/large/eval_seqBN_r4full_ds_.csv')['wer'].mean()

0.4852944095193342

In [130]:
import json

with open('/ceph/dpandya/notsofar/nsfd_adap_segments/adapter_base/large/full_ds/seqBN_r4full_ds/training_args.json', 'r') as f:
    tr_args = json.load(f)

tr_args

{'output_dir': '/ceph/dpandya/notsofar/nsfd_adap_segments/adapter_base/large/full_ds/seqBN_r4full_ds',
 'overwrite_output_dir': False,
 'do_train': False,
 'do_eval': True,
 'do_predict': False,
 'eval_strategy': 'epoch',
 'prediction_loss_only': False,
 'per_device_train_batch_size': 4,
 'per_device_eval_batch_size': 8,
 'per_gpu_train_batch_size': None,
 'per_gpu_eval_batch_size': None,
 'gradient_accumulation_steps': 2,
 'eval_accumulation_steps': None,
 'eval_delay': 0,
 'torch_empty_cache_steps': None,
 'learning_rate': 0.0005,
 'weight_decay': 0.0,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'max_grad_norm': 1.0,
 'num_train_epochs': 10,
 'max_steps': -1,
 'lr_scheduler_type': 'linear',
 'lr_scheduler_kwargs': {},
 'warmup_ratio': 0.1,
 'warmup_steps': 0,
 'log_level': 'passive',
 'log_level_replica': 'warning',
 'log_on_each_node': True,
 'logging_dir': '/ceph/dpandya/notsofar/nsfd_adap_segments/adapter_base/large/full_ds/seqBN_r4full_ds/runs/Nov12_20-09-1

In [ ]:
len(os.listdir('/ceph/dpandya/notsofar/train_set/240825.1_train/new_segmented_audios/'))

10756

In [18]:
pd.read_csv('/ceph/dpandya/notsofar/train_set/240825.1_train/train.csv')

,sc_files,gt_transcription_files,mic
0,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,plaza_0
1,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,rockfall_2
2,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,meetup_0
3,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,rockfall_1
4,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,rockfall_0
...,...,...,...
341,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,plaza_0
342,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,rockfall_2
343,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,meetup_0
344,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,rockfall_1


In [28]:
current_train = pd.read_csv('/ceph/dpandya/notsofar/nsfd_adap_segments/new_train_segments.csv')
current_train['dur'] = current_train['stop']-current_train['start']
current_train['dur'].mean()/1000

17.088404663923182

In [30]:
(len(current_train)*17.088)/3600

6.92064

In [3]:
pd.read_csv('/ceph/dpandya/notsofar/newSegments_adap/large/RF4/large_seqBN_5e.csv')

,org_audio_file,segmented_audio_file,text,prediction_large_seqBN_5e,wer_seqBN_5e
0,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/M...,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/n...,ok so were designing todays hackathon uhhuh ye...,ok so were designing todays hackathon uhhuh er...,0.160714
1,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/M...,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/n...,we obey with that it was obvious she made a mi...,but it was it was obvious she made a mistake i...,0.370787
2,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/M...,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/n...,everybody is getting together and i mean every...,and everybody is getting together and i mean e...,0.096154
3,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/M...,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/n...,if we win yeah then we get the trip that big t...,if we win yeah then we get the trip the big tr...,0.441558
4,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/M...,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/n...,this is like basically make food better techno...,this is like basic make food better technology...,0.600000
...,...,...,...,...,...
3468,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/M...,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/n...,precise perspective and not look about the ext...,precise perspective and not look about the ext...,0.132353
3469,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/M...,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/n...,yeah and when you look at the the atmosphere i...,yeah when you look at the the atmosphere in ve...,0.169014
3470,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/M...,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/n...,yeah well i mean venus has the same gravity as...,yeah well i mean venus is the same gravity as ...,0.282051
3471,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/M...,/ceph/dpandya/notsofar/dev_set/240825.1_dev1/n...,we can work we we need greenhouses in both pla...,we can work we would need greenhouses in both ...,0.282051


In [2]:
pd.read_csv('/ceph/dpandya/notsofar/newSegments_adap/large/RF4/large_seqBN_5e.csv')['wer_seqBN_5e'].mean()

0.3607927237315797

In [ ]:
#pd.read_csv('/ceph/dpandya/notsofar/train_set/240825.1_train/train.csv')
#pd.read_csv('/ceph/dpandya/notsofar/train_set/240825.1_train/segmented_audios.csv')
pd.read_csv('/ceph/dpandya/notsofar/train_set/240825.1_train/new_segmented_audios/new_segmented_audios.csv')

,org_audio_file,segmented_audio_file,text
0,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,alright everyone let us continue on with our ...
1,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,have any of you guys used it yet? um not not y...
2,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,i would i would love to try it to see. but is ...
3,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,so you don't have to have someone monitoring y...
4,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,"oh i see, oh i see, ok. and you could just to ..."
...,...,...,...
6776,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,i think both is is a good option. you think bo...
6777,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,there could be um like toys and things for kid...
6778,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,what i feel wasn't set up an ice cream cone li...
6779,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,maybe like each level like the first level is ...
